In [1]:
import matplotlib.lines as mlines
import time
import numpy as np
import scipy.sparse as sps
from tqdm import tqdm
import matplotlib.pyplot as plt


class Graph:
    
    def __init__(self, n_nodes, n_features, features, n_steps_max_live_graph = 5):
        matrix_shape = np.zeros((n_nodes,n_nodes)) #matrice

        #prob archi entranti in un nodo
        for j in tqdm(range(0,n_nodes)):
            p = np.zeros(n_nodes) #array prob archi entranti in un nodo
            parameters = np.random.binomial(1, 0.5, size=(n_nodes, n_features)) #parametri
            for i in range(0,n_nodes):
                p[i] = np.dot(features, parameters[i])
            matrix_shape[j] = p
 
        matrix = matrix_shape.T.copy()
        matrix /= np.sum(matrix, axis=0)
        np.fill_diagonal(matrix, 0.)
        self.matrix = matrix
        self.n_steps_max_live_graph = n_steps_max_live_graph
        
        
    def influence_cascade(self, n_steps_max_live_graph, temp_seeds):
        
        active_nodes = temp_seeds.copy()
        rand = np.random.rand

        newly_active_nodes = active_nodes.copy()
        t = 0
        while t < n_steps_max_live_graph and np.sum(newly_active_nodes) > 0:
            active_nodes_mask = np.array(newly_active_nodes, dtype=bool)
            p = self.matrix[active_nodes_mask]
            activated_edges = p > rand(p.shape[0], n_nodes)
            newly_active_nodes = 1 - active_nodes
            newly_active_nodes[~(np.any(activated_edges, axis=0))] = 0
            active_nodes = np.array(active_nodes + newly_active_nodes)
            t += 1

        return active_nodes
        

    def monte_Carlo_Sampling(self, n_iterations_mc, temp_seeds):
        
        nodes = np.random.binomial(0, 1.0, size=(n_nodes))
        for _ in range(n_iterations_mc):
            nodes_live_graph = self.influence_cascade(self.n_steps_max_live_graph, temp_seeds)
            nodes = np.array(nodes + nodes_live_graph)
        nodes_by_iteration = nodes / n_iterations_mc
        return np.sum(nodes_by_iteration)
    

    def greedy_algorithm(self, budget, n_steps_max_live_graph,  epsilon, delta, random=False, random_number=10):
        
        set_seeds = np.random.binomial(0, 1.0, size=(n_nodes))
        seeds_acquired = []
        prev_best = 0
        marginal_increase = -1
        if random == False:
            for _ in range(budget):
                results_monte_carlo = []
                n_seeds = len(set_seeds.nonzero()[0])
                m_c_sampling_iterations = int((1 / (epsilon ** 2)) * np.log(n_seeds + 1) * np.log(1 / delta))
                if m_c_sampling_iterations == 0:
                    m_c_sampling_iterations = 10


                for i in range(n_nodes):
                    if set_seeds[i] == 0:
                        temp_seeds = set_seeds.copy()
                        temp_seeds[i] = 1
                        results_monte_carlo.append(self.monte_Carlo_Sampling(m_c_sampling_iterations,temp_seeds))
                    else:
                        results_monte_carlo.append(0)

                best_arg = np.argmax(results_monte_carlo)
                marginal_increase = results_monte_carlo[best_arg] - prev_best

                if marginal_increase > 0:
                    obj_func_m_c.append(m_c_sampling_iterations)
                    nodes_infected.append(results_monte_carlo[best_arg])
                    prev_best = results_monte_carlo[best_arg]
                    set_seeds[best_arg] = 1
                    seeds_acquired.append(best_arg)
                    print(results_monte_carlo[best_arg])
                    print("{} node acquired".format(best_arg) )
                else:
                    print("marginal increase <= 0")
                    break

            return set_seeds, seeds_acquired
        else:
            for _ in range(budget):
                n_seeds = len(set_seeds.nonzero()[0])
                m_c_sampling_iterations = int((1 / (epsilon ** 2)) * np.log(n_seeds + 1) * np.log(1 / delta))
                if m_c_sampling_iterations == 0:
                    m_c_sampling_iterations = 10
                
                marginal_increase = -1
                t = 0
                while t!=random_number and marginal_increase < 0:
                    i = np.random.randint(0, self.matrix.shape[0])
                    if set_seeds[i] == 0:
                        temp_seeds = set_seeds.copy()
                        temp_seeds[i] = 1
                        result_mc = self.monte_Carlo_Sampling(m_c_sampling_iterations,temp_seeds)
                        marginal_increase = result_mc - prev_best
                    t+=1
                
                if marginal_increase > 0:
                    obj_func_m_c.append(m_c_sampling_iterations)
                    nodes_infected.append(result_mc)
                    prev_best = result_mc
                    set_seeds[i] = 1
                    seeds_acquired.append(i)
                    print(marginal_increase)
                    print("{} node acquired".format(i) )
                else:
                    print("marginal increase <= 0")
                    break
                    
            return set_seeds, seeds_acquired
                




In [ ]:
#Print one curve of Objective Function/Approximation Bound
n_nodes = 1000 #Number of Nodes
n_features = 4 #Number of Features
features = np.random.dirichlet(np.ones(n_features), size=1) #Features

epsilon = 0.2 
delta = 0.35 

initial_budget = 50  # Budget

random = True #Randomize search of best new seed
random_number=100

nodes_infected = []
obj_func_m_c = []

start_time = time.time()
graph = Graph(n_nodes, n_features, features)

print("Graph created in {:.2f} seconds".format(time.time()-start_time))

start_time = time.time()
seeds_final, seeds_acquired = graph.greedy_algorithm(initial_budget, n_steps_max_live_graph, epsilon, delta, random, random_number)
print(seeds_acquired)


time_algo = time.time()-start_time
print("Greedy computed in {:.2f} seconds".format(time.time()-start_time))

plt.title("Influence Maximization\n(" + str(n_nodes) + " nodes | " + str(initial_budget)
              + " budget | " + str(epsilon)
              + " epsilon | " + str(delta) + " delta)")
plt.xlabel("Approximation Bound (M.C. Repetitions) ")
plt.ylabel("Objective Function (Nodes Infected)")
plt.plot(obj_func_m_c, nodes_infected)
plt.savefig("{}n-r{}-e{}-d{}-s{}-b{}-{:.2f}sec.png".format(n_nodes,random,epsilon,delta,n_steps_max_live_graph,initial_budget,time_algo), dpi=200)
plt.show()

In [ ]:
#Print Rewards of different combinations of Delta and Epsilon
n_nodes = 1000 
n_features = 4
features = np.random.dirichlet(np.ones(n_features), size=1)

n_steps_max_live_graph = 5 
epsilon = [0.2, 0.3, 0.4, 0.5] 
delta = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8] 

initial_budget = 50  

random = True
random_number=100

nodes_infected = []
obj_func_m_c = []
rewards=[]
diff_epsilon = []



start_time = time.time()
graph = Graph(n_nodes, n_features, features,  n_steps_max_live_graph)

print("Graph created in {:.2f} seconds".format(time.time()-start_time))

for e in epsilon:
    rewards=[]
    for d in delta:
        start_time = time.time()
        seeds_final, seeds_acquired = graph.greedy_algorithm(initial_budget, n_steps_max_live_graph, e, d, random, random_number)
        print(seeds_acquired)
        
        n_seeds = len(seeds_final.nonzero()[0])
        n_iterations_mc = int((1 / (e ** 2)) * np.log(n_seeds + 1) * np.log(1 / d))
        performance = graph.monte_Carlo_Sampling(n_iterations_mc, seeds_final)
        rewards.append(performance)

        time_algo = time.time()-start_time
        print("Greedy computed in {:.2f} seconds".format(time.time()-start_time))
    diff_epsilon.append(rewards.copy())

plt.title("Influence Maximization\n(" + str(n_nodes) + " nodes | " + str(initial_budget)
              + " budget) ")
plt.xlabel("Delta")
plt.ylabel("Rewards")
red_line = mlines.Line2D([], [], color='red', label='e = 0.1')
green_line = mlines.Line2D([], [], color='green', label='e = 0.2')
blue_line = mlines.Line2D([], [], color='blue', label='e = 0.3')
yellow_line = mlines.Line2D([], [], color='yellow', label='e = 0.4')
plt.legend(handles=[red_line, green_line, blue_line, yellow_line])
plt.plot(delta, diff_epsilon[0], 'r')
plt.plot(delta, diff_epsilon[1], 'g')
plt.plot(delta, diff_epsilon[2], 'b')
plt.plot(delta, diff_epsilon[3], 'y')
plt.savefig("{}n-{}b-comparison.png".format(n_nodes, initial_budget), dpi=200)
plt.show()


In [ ]:
#Print curves of Objective Function/Approximation Bound with delta fixed and different values of epsilon
n_nodes = 1000
n_features = 4
features = np.random.dirichlet(np.ones(n_features), size=1)

n_steps_max_live_graph = 5 
epsilon = [0.1, 0.2, 0.3, 0.4] 
delta = 0.30 

initial_budget = 50  

random = True
random_number=100

nodes_infected = []
obj_func_m_c = []
tot_infected=[]
tot_obj = []

start_time = time.time()
graph = Graph(n_nodes, n_features, features,  n_steps_max_live_graph)

print("Graph created in {:.2f} seconds".format(time.time()-start_time))

for e in epsilon:
    nodes_infected = []
    obj_func_m_c = []
    
    start_time = time.time()
    seeds_final, seeds_acquired = graph.greedy_algorithm(initial_budget, n_steps_max_live_graph, e, delta, random, random_number)
    print(seeds_final)
    print(seeds_acquired)

    time_algo = time.time()-start_time
    print("Greedy computed in {:.2f} seconds".format(time.time()-start_time))
    tot_infected.append(nodes_infected)
    tot_obj.append(obj_func_m_c)
    

plt.title("Influence Maximization\n(" + str(n_nodes) + " nodes | " + str(initial_budget)
              + " budget | "  + str(delta)
              + " delta) ")
plt.xlabel("Approximation Bound(Iterations of M.C.)")
plt.ylabel("Objective Function ( Nodes Infected)")
red_line = mlines.Line2D([], [], color='red', label='e = 0.1')
green_line = mlines.Line2D([], [], color='green', label='e = 0.2')
blue_line = mlines.Line2D([], [], color='blue', label='e = 0.3')
yellow_line = mlines.Line2D([], [], color='yellow', label='e = 0.4')
plt.legend(handles=[red_line, green_line, blue_line, yellow_line])
plt.plot(tot_obj[0], tot_infected[0], 'r')
plt.plot(tot_obj[1], tot_infected[1], 'g')
plt.plot(tot_obj[2], tot_infected[2], 'b')
plt.plot(tot_obj[3], tot_infected[3], 'y')
plt.savefig("{}n-{}b-comparison-inner-greedy-epsilon.png".format(n_nodes, initial_budget), dpi=200)
plt.show()

In [ ]:
#Print curves of Objective Function/Approximation Bound with Epsilon fixed and different values of Delta
n_nodes = 1000 
n_features = 4
features = np.random.dirichlet(np.ones(n_features), size=1)

n_steps_max_live_graph = 5 
epsilon = 0.1 
delta = [0.1, 0.2, 0.3, 0.4] 

initial_budget = 50  

nodes_infected = []
obj_func_m_c = []
tot_infected=[]
tot_obj = []

random = False
random_number=100

start_time = time.time()
graph = Graph(n_nodes, n_features, features,  n_steps_max_live_graph)

print("Graph created in {:.2f} seconds".format(time.time()-start_time))

for d in delta:
    nodes_infected = []
    obj_func_m_c = []
    
    start_time = time.time()
    seeds_final, seeds_acquired = graph.greedy_algorithm(initial_budget, n_steps_max_live_graph, epsilon, d, random, random_number)
    print(seeds_final)
    print(seeds_acquired)

    time_algo = time.time()-start_time
    print("Greedy computed in {:.2f} seconds".format(time.time()-start_time))
    tot_infected.append(nodes_infected)
    tot_obj.append(obj_func_m_c)
    

plt.title("Influence Maximization\n(" + str(n_nodes) + " nodes | " + str(initial_budget)
              + " budget | " + str(epsilon)
              + " epsilon) ")
plt.xlabel("Approximation Bound(Iterations of M.C.)")
plt.ylabel("Objective Function ( Nodes Infected)")
red_line = mlines.Line2D([], [], color='red', label='d = 0.1')
green_line = mlines.Line2D([], [], color='green', label='d = 0.2')
blue_line = mlines.Line2D([], [], color='blue', label='d = 0.3')
yellow_line = mlines.Line2D([], [], color='yellow', label='d = 0.4')
plt.legend(handles=[red_line, green_line, blue_line, yellow_line])
plt.plot(tot_obj[0], tot_infected[0], 'r')
plt.plot(tot_obj[1], tot_infected[1], 'g')
plt.plot(tot_obj[2], tot_infected[2], 'b')
plt.plot(tot_obj[3], tot_infected[3], 'y')
plt.savefig("{}n-{}b-comparison-inner-greedy-delta.png".format(n_nodes, initial_budget), dpi=200)
plt.show()